# Face Mask Detection using MobileNet

In [ ]:
#Defining our data generators
from __future__ import print_function
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
import os

img_rows, img_cols = 128,128
batch_size = 16

train_data_dir = './dataset/train'
validation_data_dir = './dataset/validation'

# Let's use some data augmentation and define our generators
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=30,
      shear_range=0.3,
      zoom_range=0.3,
      width_shift_range=0.4,
      height_shift_range=0.4,
      horizontal_flip=True,
      fill_mode='nearest')
 
validation_datagen = ImageDataGenerator(rescale=1./255)
 
train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=True)

validation_generator = validation_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=True)

train_images = train_generator.samples
validation_images = validation_generator.samples

# Loading the MobileNet

In [ ]:
"""
    We will use the pretrained MobileNet to which we will add our own network of FC layers.
    We then train the new model formed for the additional layers.

"""
from keras.applications import MobileNet

# Lets load the MobileNet model without the top or FC layers
MobileNet = MobileNet(weights = 'imagenet', 
                 include_top = False, 
                 input_shape = (img_rows, img_cols, 3))

# Layers are set to be trainable as True by default but lets make them untrainable
for layer in MobileNet.layers:
    layer.trainable = False
    
# Let's check our layers 
for (i,layer) in enumerate(MobileNet.layers):
    print(str(i) + " "+ layer.__class__.__name__, layer.trainable)

In [ ]:
def addTopModelMobileNet(bottom_model):
    """
    
        Creates the head of the model that will be 
        placed ontop of the bottom layers
    
    """

    top_model = bottom_model.output
    top_model = GlobalAveragePooling2D()(top_model)
    top_model = Dense(512,activation='relu')(top_model)
    top_model = Dense(128,activation='relu')(top_model)
    top_model = Dense(64,activation='relu')(top_model)
    
    top_model = Dense(2,activation='sigmoid')(top_model)
    return top_model


#Combining the model
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, GlobalAveragePooling2D
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.models import Model

FC_Head = addTopModelMobileNet(MobileNet)

model = Model(inputs = MobileNet.input, outputs = FC_Head)

print(model.summary())

# Training the model

In [ ]:
#Training our model
#One could do some tweakings!
from keras.optimizers import RMSprop, SGD,Adadelta
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

                     
checkpoint = ModelCheckpoint("face_mask_detector.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)

earlystop = EarlyStopping(monitor = 'val_loss', 
                          min_delta = 0, 
                          patience = 3,
                          verbose = 1,
                          restore_best_weights = True)

reduce_lr = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.1, patience = 2, verbose = 1, min_delta = 0.0001)

callbacks = [earlystop, checkpoint, reduce_lr]

nb_train_samples = train_images
nb_validation_samples = validation_images
batch_size = 16
epochs = 20

model.compile(loss = 'categorical_crossentropy',
                  optimizer = 'rmsprop',
                  metrics = ['accuracy'])

history = model.fit_generator(
    train_generator,
    steps_per_epoch = nb_train_samples // batch_size,
    epochs = epochs,
    callbacks = callbacks,
    validation_data = validation_generator,
    validation_steps = nb_validation_samples // batch_size)


# Testing the model

In [2]:
# Loading our model
from keras.models import load_model
classifier = load_model('face_mask_detector.h5')

In [3]:
#Test on realtime video
from os import listdir
from os.path import isfile, join
import os
import cv2
import numpy as np
img_rows, img_cols = 128,128
# A function that puts the predicted class lables on the parametric image frames
def draw_label(image, point, label, font=cv2.FONT_HERSHEY_SIMPLEX,
               font_scale=0.8, thickness=1):
    size = cv2.getTextSize(label, font, font_scale, thickness)[0]
    x, y = point
    cv2.rectangle(image, (x, y - size[1]), (x + size[0], y), (255, 0, 0), cv2.FILLED)
    cv2.putText(image, label, point, font, font_scale, (255, 255, 255), thickness, lineType=cv2.LINE_AA)
    
#Define our prediction dictionary
face_classes = {0: 'with_mask', 1: 'without_mask'}
img_size = 128

detector = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

cap = cv2.VideoCapture(0)

while True:
    
    ret, frame = cap.read()
    preprocessed_faces = []           
 
    input_img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    img_h, img_w, _ = np.shape(frame)
    detected = detector.detectMultiScale(frame)
    faces = np.empty((len(detected), img_size, img_size, 3))
   
    if len(detected) > 0:
        
        for f in detected:
            
            # Obtain the coordinates of the detected face and draw a bounding box
            x1, y1, w, h = [v for v in f]
            cv2.rectangle(frame, (x1, y1), (x1+w,y1+h), (255, 0, 0), 2)
            face =  frame[y1:y1+h, x1:x1+w, :]
            face = cv2.resize(face, (img_rows,img_cols))
            face = face.astype("float32") / 255.0
            face = np.expand_dims(face, axis=0)
            preprocessed_faces.append(face)

        
        # Make predictions for the detected face 
        face_labels = []
        for i, d in enumerate(detected):
            preds = classifier.predict(preprocessed_faces[i])[0]
            face_labels.append(face_classes[np.argmax(preds,axis=0)])
#             print(preds)
        
        # Putting labels on frames
        for i, d in enumerate(detected):
            label = "{}".format(face_labels[i])
#             print(label)
            draw_label(frame, (x1,y1), label)
    
    #Display the results
    cv2.imshow("Face Mask Recognition", frame)
    if cv2.waitKey(1) == 13: #13 is the Enter Key
        break

cap.release()
cv2.destroyAllWindows()      

In [8]:
#You can use some images for testing also using this part of the code
from os import listdir
from os.path import isfile, join
import os
import cv2
import numpy as np

# A function that puts the predicted class lables on the parametric image frames
def draw_label(image, point, label, font=cv2.FONT_HERSHEY_SIMPLEX,
               font_scale=0.8, thickness=1):
    size = cv2.getTextSize(label, font, font_scale, thickness)[0]
    x, y = point
    cv2.rectangle(image, (x, y - size[1]), (x + size[0], y), (255, 0, 0), cv2.FILLED)
    cv2.putText(image, label, point, font, font_scale, (255, 255, 255), thickness, lineType=cv2.LINE_AA)
    
#Define our prediction dictionary
face_classes = {0: 'with_mask', 1: 'without_mask'}
img_size = 128

detector = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    
frame = cv2.imread('E:\Others\Shimla Manali\IMG_20210910_134647.jpg')
frame = cv2.resize(frame,(1080,720))
preprocessed_faces = []           

input_img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
img_h, img_w, _ = np.shape(frame)
detected = detector.detectMultiScale(frame)
faces = np.empty((len(detected), img_size, img_size, 3))

if len(detected) > 0:

    for f in detected:

        # Obtain the coordinates of the detected face and draw a bounding box
        x1, y1, w, h = [v for v in f]
        cv2.rectangle(frame, (x1, y1), (x1+w,y1+h), (255, 0, 0), 2)
        face =  frame[y1:y1+h, x1:x1+w, :]
        face = cv2.resize(face, (img_rows,img_cols))
        face = face.astype("float32") / 255.0
        face = np.expand_dims(face, axis=0)
        preprocessed_faces.append(face)


    # Make predictions for the detected face 
    face_labels = []
    for i, d in enumerate(detected):
        preds = classifier.predict(preprocessed_faces[i])[0]
        face_labels.append(face_classes[np.argmax(preds,axis=0)])
        print(preds)

    # Putting labels on frames
    for i, d in enumerate(detected):
        label = "{}".format(face_labels[i])
        print(label)
        draw_label(frame, (x1,y1), label)

# Display the results
cv2.imshow("Face Recognition", frame)
cv2.waitKey(0) == 13


cap.release()
cv2.destroyAllWindows()      

[2.2908185e-05 9.9999905e-01]
[0. 1.]
[0. 1.]
without_mask
without_mask
without_mask
